# Fluent Python


Reading fluent python by Luciano Ramalho this is a collection of my favourite bits.


### Python data model - bool and `__bool__`


The python data model allows you to implement special methods that make custom types behave like the builtin types.


In [1]:
class Example:
    def __len__(self):
        print(f"running __len__")
        return 10

    def __repr__(self):
        print(f"running __repr__")
        return f"Example()"

    def __str__(self):
        print(f"running __str__")
        return f"<Example Python object"

    def __bool__(self):
        print(f"running __bool__")
        return True


In [2]:
example = Example()
example


running __repr__


Example()

In [3]:
len(example)


running __len__


10

In [4]:
if example:
    pass


running __bool__


To check the whether `example` is truthy it calls effectively calls `bool(example)` which looks for `example.__bool__()`
If there is no `__bool__` python tries to invoke `__len__` and if that returns 0 then `False` else `True`.


### `__repr__` and `__str__`


`__repr__` - unambiguous for developers.  
`__str__` - for the end user.


### Unpacking sequences


In [5]:
person_data = ("Simon", "Ward-Jones", 1, 2, 3)
name, surname, *numbers = person_data
print(name, surname, numbers, sep="\n")


Simon
Ward-Jones
[1, 2, 3]


In [6]:
# Unpacking into a list
person_data = [*person_data]
person_data


['Simon', 'Ward-Jones', 1, 2, 3]

### Bad matrix


In [7]:
matrix = [[None] * 3] * 3
matrix[0][0] = 1
matrix


[[1, None, None], [1, None, None], [1, None, None]]

In [8]:
# you probably want:
matrix = []
for _ in range(3):
    matrix.append([None] * 3)
matrix[0][0] = 1
matrix


[[1, None, None], [None, None, None], [None, None, None]]

### Key in min, max and sorted


In [9]:
numbers = [3, "5", 1, "10", 2, "4", "6"]


In [10]:
print(sorted(numbers, key=int))
print(min(numbers, key=int))


[1, 2, 3, '4', '5', '6', '10']
1


### Unpacking mappings


In [12]:
person_data_1 = {"name": "Simon", "surname": "Ward-Jones", "age": 20}
person_data_2 = {
    "name": "Simon",
    "surname": "Ward-Jones",
    "age": 30,
    "height": 2,
    "weight": 3,
}


In [13]:
{**person_data_1, **person_data_2}


{'name': 'Simon', 'surname': 'Ward-Jones', 'age': 30, 'height': 2, 'weight': 3}

In [14]:
person_data_1 | person_data_2


{'name': 'Simon', 'surname': 'Ward-Jones', 'age': 30, 'height': 2, 'weight': 3}

### Dictionary comprehensions


In [16]:
country_population_pairs = [
    ("China", 1409517397),
    ("India", 1324171354),
    ("United States", 325234000),
    ("Indonesia", 255461563),
    ("Brazil", 210147125),
    ("Pakistan", 197015955),
    ("Nigeria", 206139589),
    ("Bangladesh", 164895000),
    ("Russia", 146585181),
    ("Japan", 126470000),
]


In [17]:
# create a dictionary of country populations
country_populations = {
    country: population for country, population in country_population_pairs
}


### Dictionary setdefault method


In [18]:
person_data_1 = {"name": "Simon", "surname": "Ward-Jones", "age": 20}
person_data_2 = {
    "name": "Simon",
    "surname": "Ward-Jones",
    "age": 30,
    "height": 2,
    "weight": 3,
}


In [19]:
# set default looks up the key and returns the value if it exists
# if it doesn't exist, it sets the key to the default value and returns the default value
person_data_1.setdefault("name", "John")


'Simon'

In [20]:
# This is really useful for something like this:
person_data_1.setdefault("animals", {}).setdefault("cats", []).append("cat")
person_data_1


{'name': 'Simon',
 'surname': 'Ward-Jones',
 'age': 20,
 'animals': {'cats': ['cat']}}

### Dictionary views and mappings work like sets


In [21]:
person_data_1 = {"name": "Simon", "surname": "Ward-Jones", "age": 20}
person_data_2 = {
    "name": "Simon",
    "surname": "Ward-Jones",
    "age": 30,
    "height": 2,
    "weight": 3,
}
# finding common keys
person_data_1.keys() & person_data_2.keys()


{'age', 'name', 'surname'}

### Unicode names


In [22]:
from unicodedata import name
import random


In [23]:
n = 0
while n < 5:
    unicode_point = chr(random.randint(0, 0x10FFFF))  # pick a random unicode point
    if unicode_name := name(unicode_point, ""):
        print(unicode_point, unicode_name)  # print the point and the name
        n += 1


𘮴 KHITAN SMALL SCRIPT CHARACTER-18BB4
潚 CJK UNIFIED IDEOGRAPH-6F5A
𝤍 SIGNWRITING STRIKE BETWEEN
ﶢ ARABIC LIGATURE TEH WITH KHAH WITH ALEF MAKSURA FINAL FORM
ם HEBREW LETTER FINAL MEM


### string translation


In [24]:
translation = str.maketrans("simon", "abcde")


In [25]:
"hello simon".translate(translation)


'helld abcde'

### Dataclasses save you a lot of typing


In [27]:
from dataclasses import dataclass
from dataclasses import field
from typing import Optional


@dataclass
class Book:
    title: str
    author: Optional[str] = "unknown"
    pages: int = 0
    price: float = field(default=0.0, repr=False)

    def __post_init__(self):
        self.long_title = self.title + " by " + self.author


In [28]:
book = Book(title="Python Programming", author="Simon Ward-Jones", pages=300)
book


Book(title='Python Programming', author='Simon Ward-Jones', pages=300)

### Variables are not boxes 📦 🏷


In [29]:
variable = ["📦 ", "🏷"]
other_variable = variable
other_variable.append("📦")
variable  # say what...


['📦 ', '🏷', '📦']

### Copy is only one level deep


In [5]:
data = [1, 2, [3, 4]]
new_data = data.copy()
for item, new_item in zip(data, new_data):
    print(id(item), id(new_item))


4560093424 4560093424
4560093456 4560093456
4685638528 4685638528


### Watch out for passing a mutable object to a function


In [14]:
# the function will modify the mutable data passed to it
data = [1, 2, 3, 4]


def sum_all_but_last(data):
    last_item = data.pop()
    print(f"Removing last item before sum: {last_item}")
    return sum(data)


data_sum_all_but_last = sum_all_but_last(data)
print(f"The sum is {data_sum_all_but_last} but data is now: {data}")


Removing last item before sum: 4
The sum is 6 but data is now: [1, 2, 3]


### Really watch out for mutable default arguments


In [16]:
class Person:
    def __init__(self, name, pets=[]):
        self.name = name
        self.pets = pets


simon = Person("Simon")
paul = Person("Paul")
paul.pets.append("cat")
# Now simon has a pet cat too!
simon.pets


['cat']

### Classic late binding lambdas trick


In [23]:
# The value of i is looked up at execution time.
for function in [lambda x: x + i for i in range(3)]:
    print(function(1))


3
3
3


### Tuples are immutable but their contents can be changed


In [27]:
address = "42 Wallaby Way, Sydney"
rooms = ["kitchen", "living room", "dining room", "bathroom"]
house = (address,)


In [28]:
house[1].append("office")
house


('42 Wallaby Way, Sydney',
 ['kitchen', 'living room', 'dining room', 'bathroom', 'office'])

### The operator module is handy when reducing


In [2]:
from operator import mul
from functools import reduce

numbers = [3, 4, 5, 6]
product_of_numbers = reduce(mul, numbers)

# without the mul operator:
# product_of_numbers = reduce(lambda x, y: x * y, numbers)

print(product_of_numbers)


360


### Use the operator itemgetter to sort a list by specific item


In [24]:
from operator import itemgetter

sorted([("a", 2), ("b", 1), ("c", 3)], key=itemgetter(1))

# This is the same as doing:
# sorted([("a", 2), ("b", 1), ("c", 3)], key=lambda x: x[1])


[('b', 1), ('a', 2), ('c', 3)]

### Use the operator attrgetter to sort by specific (maybe nested) attribute


In [25]:
from operator import attrgetter
from typing import NamedTuple, Optional


class Pet(NamedTuple):
    name: str
    species: str
    age: int


class Human(NamedTuple):
    name: str
    pet: Pet


people = [
    Human(name="Simon", pet=Pet(name="Whiskers", species="cat", age=3)),
    Human(name="Richard", pet=Pet(name="Giles", species="shark", age=1)),
    Human(name="Paul", pet=Pet(name="Alfie", species="dog", age=2)),
    Human(name="Tom", pet=Pet(name="Rupert", species="cat", age=3)),
]


In [26]:
# sort by animal age ascending
sorted(people, key=attrgetter("pet.age"))

# This is equivalent to:
# sorted(people, key=lambda x: x.pet.age)


[Human(name='Richard', pet=Pet(name='Giles', species='shark', age=1)),
 Human(name='Paul', pet=Pet(name='Alfie', species='dog', age=2)),
 Human(name='Simon', pet=Pet(name='Whiskers', species='cat', age=3)),
 Human(name='Tom', pet=Pet(name='Rupert', species='cat', age=3))]

In [27]:
# sort by animal age then by owner name (note Simon's 3 year olf pet comes before Tom's)
sorted(people, key=attrgetter("pet.age", "name"))

# This is equivalent to:
# sorted(people, key=lambda x: (x.pet.age, x.name))


[Human(name='Richard', pet=Pet(name='Giles', species='shark', age=1)),
 Human(name='Paul', pet=Pet(name='Alfie', species='dog', age=2)),
 Human(name='Simon', pet=Pet(name='Whiskers', species='cat', age=3)),
 Human(name='Tom', pet=Pet(name='Rupert', species='cat', age=3))]

### Key word only arguments


In [51]:
def count_letters(string: str, *, letters: str = "aeiou") -> int:
    return sum(1 for c in string if c.lower() in letters)


# This call would fail:
# count_letters("simon", "aei")
# One must call with letters key word like:
count_letters("simon", letters="aei")


1

### Positional only parameters


In [52]:
def count_letters(string: str, /, letters: str = "aeiou") -> int:
    return sum(1 for c in string if c.lower() in letters)


# This call would fail:
# count_letters(string="simon", letters="aei")
# One must call using positional argument like:
count_letters("simon", "aei")


1

### Positional only parameters and Keyword only parameters


In [50]:
# Note we can combine positional only with keyword only arguments


def count_letters(
    string: str, /, letters: str = "aeiou", *, case_sensitive: bool = False
) -> int:
    if not case_sensitive:
        string = string.lower()
    return sum(1 for c in string if c.lower() in letters)


# In this case:
#     string must be a positional argument
#     letters can be a keyword argument or a positional argument
#     case_sensitive mst be a keyword argument

count_letters("simon", letters="aei", case_sensitive=True)
count_letters("simon", "aei", case_sensitive=True)


1

### Closures capture variable values in the outer scope


In [39]:
def get_wife_function():
    wives = ()

    def add_wife(name: str):
        nonlocal wives
        wives += (name,)
        return wives

    return add_wife


henrys_wives = get_wife_function()

henrys_wives("catherine_of_aragon")
henrys_wives("anne_boleyn")
henrys_wives("jane_seymour")
henrys_wives("anne_of_cleves")
henrys_wives("catherine_howard")
wives = henrys_wives("catherine_parr")


print(henrys_wives.__code__.co_varnames)
print(henrys_wives.__code__.co_freevars)

print(wives)
print(henrys_wives.__closure__[0].cell_contents)


('name',)
('wives',)
('catherine_of_aragon', 'anne_boleyn', 'jane_seymour', 'anne_of_cleves', 'catherine_howard', 'catherine_parr')
('catherine_of_aragon', 'anne_boleyn', 'jane_seymour', 'anne_of_cleves', 'catherine_howard', 'catherine_parr')


### Another example of scope to create a simple counter


In [44]:
# We specify non local to reference a variable that is in the outer scope.
# If we did not we would get an error.


def get_counter():
    count = 0

    def counter():
        nonlocal count
        count += 1
        return count

    return counter


counter = get_counter()

while (x := counter()) < 5:
    print(x)


1
2
3
4


### Use single dispatch to implement polymorphism


In [20]:
# The function called depends on the type at runtime.

from typing import NamedTuple
from collections.abc import Iterable
from functools import singledispatch
import re


class RGB(NamedTuple):
    red: int
    green: int
    blue: int


@singledispatch
def get_hex(rgb: Iterable[int]) -> str:
    print("Using str implementation")
    red, green, blue = rgb
    return f"#{red:02x}{green:02x}{blue:02x}"


@get_hex.register
def _(rgb: RGB) -> str:
    print("Using RGB implementation")
    return f"#{rgb.red:02x}{rgb.green:02x}{rgb.blue:02x}"


@get_hex.register
def _(rgb: str) -> str:
    red, green, blue = (int(x) for x in re.findall(r"\d+", rgb))
    print("Using generic Iterator implementation")
    return f"#{red:02x}{green:02x}{blue:02x}"


print(get_hex("RGB(10, 20, 30)"))
print(get_hex(RGB(red=10, green=20, blue=30)))
print(get_hex((10, 20, 30)))


Using generic Iterator implementation
#0a141e
Using RGB implementation
#0a141e
Using str implementation
#0a141e


# Fin.
